In [1]:
import random
import numpy as np
from classes.helper import Helper
import random
import os
from algorithms.pies import pies_sampling
from algorithms.ties import ties_sampling
from algorithms.DLAS import dlas_algorithm
from algorithms.DUE import due_sampling_directed
from algorithms.FF import forest_fire_model
from algorithms.icla_ns import icla_ns_directed
from algorithms.EDLAS import edlas_sampling
from classes.dataset_manager import DatasetManager
from classes.benchmark import BenchmarkStatic
import json
random.seed(42)
np.random.seed(42)

In [ ]:
filename = 'soc-redditHyperlinks-title.csv'
manager = DatasetManager(folder_path='data',file_extension=filename)
datasets = manager.load_edgelists(source_col='SOURCE', target_col='TARGET', timestamp_col='TIMESTAMP')
datasets

In [ ]:
G = manager.graphs[filename]
helper = Helper()
fractions = [0.1,0.2,0.3]
St = [
    [
        pies_sampling(G=G, phi=fraction),
        ties_sampling(G=G, phi=fraction),
        dlas_algorithm(graph=G, sample_ratio=fraction, num_iterations=10000),
        due_sampling_directed(graph=G, sampling_ratio=fraction)[-1],
        forest_fire_model(graph=G, min_percent=fraction, p_f=0.2, p_b=0.1)[-1],
        icla_ns_directed(graph=G,f=fraction, tau=0.05),
        edlas_sampling(graph=G, sampling_ratio=fraction)
    ][:]
    for fraction in fractions[:]
]

In [ ]:
benchmark = BenchmarkStatic(G)
path = f'results/base_statistics {filename.split(".")[0]}.json'
if not os.path.exists(path):
    results  = benchmark.base_statistics()
    print(results)
    with open(f'results/base_statistics {filename.split(".")[0]}.json', 'w') as f:
        json.dump(results, f)

In [ ]:
order_of_sampling = ["PIES", "TIES", "DLAS", "DUE", "FF", "ICLA", "EDLAS"]
bench_list = {}
benchmark.precompute()

In [ ]:
for fraction,  st_t in zip(fractions, St):
    for sampling_technique, graph_sampled in zip(order_of_sampling, st_t):
        benchmark.Gs = graph_sampled
        bench_dict = {}
        print(f"-- New Benchmark method: {sampling_technique} fraction: {fraction} --")
        s1 = benchmark.S1()
        s2 = benchmark.S2()
        s3 = benchmark.S3()
        s4 = benchmark.S4()
        s5 = benchmark.S5()
        s6 = benchmark.S6()
        s7,s8 = benchmark.S7_S8()
        s9 = benchmark.S9()
        bench_dict = {"S1": s1, "S2": s2, "S3": s3, "S4": s4, "S5": s5, "S6": s6, "S7": s7, "S8": s8, "S9": s9}
        bench_list[f"{round(fraction,5)}_{sampling_technique}"] = bench_dict
        # print(f"S1: {s1}\nS2: {s2}\nS3: {s3}\nS4: {s4}\nS5: {s5}\nS6: {s6}\nS7: {s7}\nS8: {s8}\nS9: {s9}")
with open(f'results static {filename.split(".")[0]}.json', "w") as f:
    json.dump(bench_list, f,indent=4)

In [7]:
with open(f'results static {filename.split(".")[0]} seed 42.json', "w") as f:
    json.dump(bench_list, f,indent=4)